# Ground Truth でカスタムのテキストラベリングタスクを実施する

このノートブックでは，Ground Truth を使った，テキストを用いた簡単なカスタムタスクを実際に試してみます．その際に必要な Lambda や jekyll テンプレートについても確認します．

## ポリシーの準備

IAM 管理ページ に移動して，ラベリングジョブを行うユーザーに対して，以下のポリシーを付与してください．この作業は，あらかじめハンズオンの前に管理者側で実施しておくことを推奨します．

- AmazonCognitoPowerUser ポリシー
- AWSLambdaFullAccess ポリシー

## ラベリング対象データの準備

ラベル付けを行う対象データを確認してから，S3 にアップロードしましょう．テキストデータの場合，manifest.json という名前のファイルに，以下のような形式でラベリング対象のテキストを記述します．この各行のデータが，あとで説明する pre-process 用の Lambda の入力として引き渡されます．

In [8]:
!cat manifest.json

{"source": "セリフ1"}
{"source": "セリフ2"}
{"source": "セリフ3"}
{"source": "あああ"}
{"source": "いいい"}
{"source": "せりふ1"}


続いてデータを S3 にアップロードします．SageMaker のファイルアップロード用のユーティリティ関数を使います．アップロード前に，**JOB_NAME の XXX を適当な 3 桁の数字に変更**してください．

In [9]:
import sagemaker
import os

BUCKET_NAME = sagemaker_session.default_bucket()
PATH = 'data/sentiment'
JOB_NAME = 'test-text-job-XXX'
FILE_NAME = 'manifest.json'

sagemaker_session = sagemaker.Session()
sagemaker_session.upload_data(
        path=FILE_NAME,
        key_prefix=os.path.join(JOB_NAME, PATH))

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-666254511816


's3://sagemaker-us-east-1-666254511816/test-text-job-001/data/text/manifest.json'

## Lambda 関数の作成

続いて Lambda 関数を 2 つ作成します．これは，ラベリングジョブの前処理と後処理を行うためのものです．それぞれの役割は以下の通りです．またそれぞれの Lambda 関数の入出力データの形式は，[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates-step3.html)を参照ください．

- 前処理: manifest.json から必要な内容を読み込んで，jekyll テンプレートに引き渡す役割
- 後処理: ラベリングされた各ワーカーの結果を集約する役割

### 前処理

前処理に使うスクリプトの中身は，以下の通りです．

In [10]:
!cat src/ground-truth-preprocess.py

import json

def lambda_handler(event, context):
    return {
        "taskInput": {
            "topic": "日常会話",
            "conversation" : event["dataObject"]["source"]
        }
    }


また，上記　Lambda に引き渡されるインプットとアウトプットの形式は，それぞれ以下の通りになります

#### インプット

`labelingJobArn` は Ground Truth 側が自動で生成する，ラベリングジョブの ARN になります．また `manifest.json` 内に記述されたラベリング対象データが，1 行ずつパースされて `dataObject` の中に埋め込まれ，Lambda 側に引き渡される形になります．

```
{
    "version": "2018-10-16",
    "labelingJobArn": <labelingJobArn>
    "dataObject" : {
        "source": "セリフ1"
    }
}
```

#### アウトプット

Lambda の出力結果は，下で示すテンプレートに引き渡されます．出力形式は JSON にする必要があります．上の Lambda では，マニフェストファイルから引き渡された会話情報を `conversation` に，また会話トピック名を新たに `topic` に入れています．これをテンプレート側で，`task.input.conversation` および `task.input.topic` で取り出すことができます．

```
{
    "taskInput": {
        "topic": "日常会話",
        "conversation" : "セリフ1"
    }
}
```

### 後処理

前処理に使うスクリプトの中身は，以下の通りです．

In [11]:
!cat src/ground-truth-postprocess.py

import json
import boto3
from urlparse import urlparse

def lambda_handler(event, context):
    consolidated_labels = []

    parsed_url = urlparse(event['payload']['s3Uri']);
    s3 = boto3.client('s3')
    textFile = s3.get_object(Bucket = parsed_url.netloc, Key = parsed_url.path[1:])
    filecont = textFile['Body'].read()
    annotations = json.loads(filecont);
    
    for dataset in annotations:
        for annotation in dataset['annotations']:
            new_annotation = json.loads(annotation['annotationData']['content'])
            label = {
                'datasetObjectId': dataset['datasetObjectId'],
                'consolidatedAnnotation' : {
                'content': {
                    event['labelAttributeName']: {
                        'workerId': annotation['workerId'],
                        'annnotationScore': new_annotation,
                        'source': dataset['dataObject']
                        }
                    }
                }
            }

同様に，引き渡されるインプットとアウトプットは，それぞれ以下の形になります．

#### インプット

`labelingJobArn` は先ほどと同様で，`roleArn` はラベリングジョブに付与した IAM ロールです．アノテーションの結果は，`payload` 内の `s3Uri` に格納された S3 オブジェクトに書かれているので，Lambda 関数内で boto3 を使ってオブジェクトを読み込んで，結果の後処理を行う形になります．

```
{
    "version": "2018-10-16",
    "labelingJobArn": "arn:aws:sagemaker:XXXX",
    "outputConfig":"s3://BUCKET_NAME/JOB_NAME/annotations",
    "labelAttributeName": "test-text-job-XXX",
    "roleArn" : "arn:aws:iam::XXXXXXXXXX",
    "payload": {
        "s3Uri": "s3://BUCKET_NAME/JOB_NAME/annotations/annotations/consolidated-annotation/consolidation-request/iteration-1/2019-02-15_01:26:56.json"
    }
 }
```

#### アウトプット

出力としては，Lambda で返された中身がそのまま配列として JSON 型で書き出されます．

```
[
  {
    "datasetObjectId": "3",
    "consolidatedAnnotation": {
      "content": {
        "test-text-job-XXX": {
          "workerId": "private.us-east-1.WQBQ5GUUOR4JHRSNUKSF43RAUM",
          "annnotations": {
            "sentiment": {
              "label": "5 - Positive"
            }
          },
          "source": {
            "content": "最悪だ"
          }
        }
      }
    }
  },
  {
    "datasetObjectId": "5",
    ...
  }
]
```

[Lambda のマネジメントコンソール](https://console.aws.amazon.com/lambda/home?region=us-east-1)を開いて，それぞれ以下の形で作成してください．Name, Runtime, Role を設定したら `Create function` を選択して，次のページで Function code の中身をコピーする形になります．その上で，右上の `Save` ボタンを押してください．

#### 前処理

- Name: sagemaker-custom-text-preprocess
- Runtime: Python 3.7
- Role: arn:aws:iam::aws:policy/AWSLambdaExecute ポリシーを付与した Role
- Function code: ground-truth-preprocess.py の内容をコピーして貼り付け

#### 後処理

- Name: sagemaker-custom-text-postprocess
- Runtime: Python 2.7
- Role: `Create a new role from one or more templetes` を選択して，`Amazon S3 object read-only permissions` を選ぶ．ロール名は適当なものをつける
- Function code: ground-truth-preprocess.py の内容をコピーして貼り付け

### テンプレートの確認

以下のコマンドを実行して，カスタムタスク用のテンプレートの中身を確認します．ここでは，Ground Truth で定義されている要素を活用して，Jekyll 向けのテンプレートを構成しています．定義されている HTML 要素の一覧については，[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-reference.html)を参照してください．機械学習に関するものとしては，例えば以下のような特定タスク向けのタグがあります．これらのタグは，crowd-form の子要素となっているため，入力結果を書き出すことができます．書き出した結果は S3 に保存されるため，S3 キー名を後処理の Lambda で受け取って，ファイル自体を読み込んで複数ワーカーの結果をマージする形になります．

- [crowd-classifier](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-classifier.html)
- [crowd-image-classifier](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-image-classifier.html)
- [crowd-bounding-box](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-bounding-box.html)
- [crowd-semantic-segmentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-semantic-segmentation.html)

またそれ以外にも，下記のようなタグも入力部品としてお使いいただくことが可能です．こちらの入力結果も，同様に後処理の Lambda に引き渡されます．

- [crowd-input](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-input.html)
- [crowd-slider](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-slider.html)
- [crowd-button](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-button.html)


このテンプレートの中で書かれている，task.input.conversation といったパラメタは，前処理用の Lambda から引き渡されるものになります．また後処理の Lambda の処理とも合わせて，このあたりの詳細については，[マニュアル](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates-step2-demo1.html)を参照してください．


In [12]:
!cat src/custom-task.templete

<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
    <crowd-classifier
      name="relevance"
      categories="['5 - ポジティブ', '4', '3', '2', '1 - ネガティブ']"
      header="次のトピックについて，提示された文章のポジネガを判定してください: {{ task.input.topic }}"
    >
      <classification-target>
        {{ task.input.conversation }}
      </classification-target>
      
      <full-instructions header="Conversation Relevance Instructions">
        <h2>提示された文章のポジネガを判定してください</h2>
      </full-instructions>

      <short-instructions>
        提示された文章のポジネガを判定してください
      </short-instructions>
    </crowd-classifier>
</crowd-form>

## ラベリングジョブの作成

まず，以下のコマンドを実行して，必要な値を生成します．

In [15]:
print('Job name: {}'.format(JOB_NAME))
print('Input data location: s3://{}/{}/{}/manifest.json'.format(
    sagemaker_session.default_bucket(), JOB_NAME, PATH))
print('Output data location: s3://{}'.format(sagemaker_session.default_bucket()))

Job name: custom-job-XXX
Input data location: s3://sagemaker-us-east-1-666254511816/custom-job-XXX/data/sentiment/manifest.json
Output data location: s3://sagemaker-us-east-1-666254511816


[SageMaker コンソールのラベリングジョブページ](https://console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-jobs) に移動して，`Create labelling job ` ボタンをクリックします．その後以下のように入力を行ってから，「Next」ボタンを押してください．

- Job name: 上の値をそのままコピー
- Input dataset location: 上の値をそのままコピー
- Output dataset location: 上の値をそのままコピー
- IAM role: 既存の SageMakerFullAccess ポリシーを付与したロールを使用

その上で，Task type として Custom を選択したら Next を押してください．


次のページでは，順番に以下のように選択してください．

- Worker types: Private を選択
- Private teams: 先ほど作成したチームを選択

その上で，Custom labeling task setup について，以下のように追加情報の入力を行います．

- Templete: Custom
- Templete code: src/custom-task.templete の中身をそのままコピペする

また一番下の Lambda 関数を指定する箇所は，以下のようにします．

- Pre-labeling task Lambda function: sagemaker-custom-text-preprocess
- Post-labeling task Lambda function: sagemaker-custom-text-postprocess

これらが終わったら Submit ボタンを押してください．

![labelling_image](./img/img001.jpg)

## ラベリングジョブの実施

あとは，先ほどの Worker 側作業ページを開いて待っていると，センチメント評価タスクが積まれます．実際にラベリングタスクを試してみてください．

![labelling_image](./img/img002.jpg)


## 結果の確認

ジョブが終了したら，マネジメントコンソール内のラベリングジョブの一覧から，実行したジョブを選択して，Output dataset location をクリックしてください．アウトプットフォルダは，以下のような階層構造になっています．

- s3://PATH/TO/JOB/OUTPUT/activelearning: 自動ラベリング機能を使った時に，その結果を格納するディレクトリです
- s3://PATH/TO/JOB/OUTPUT/annotations: 人手によるラベリングジョブの結果が格納されるディレクトリです．個々人のラベリング結果を入れる worker-responce サブディレクトリ，各バッチごとのアノテーション結果をまとめた consolidated-annotation サブディレクトリ，バッチ内の判定結果をマニフェストファイルの形に落とした intermediate サブディレクトリの 3 つが含まれます
- s3://PATH/TO/JOB/OUTPUT/inference: 自動ラベリング機能の実施時の，バッチ推論ジョブの入出力データが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/manifests: 最終結果のマニフェストファイルが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/training: 自動ラベリング機能の実施時の，学習ジョブに関するデータが格納されるディレクトリです

上記以外の詳細な説明は[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-output.html)をご覧ください．

では，最終的なラベリング結果を取得して中身を確認しましょう．下記コマンドを実行して，中身を確認します．

In [ ]:
import sys

s3.download_file(BUCKET_NAME, '{}/manifests/output/output.manifest'.format(JOB_NAME), 'output.manifest')

with open('output.manifest', 'r') as f:
    for line in f.readlines():
        print(line)